# Здравствуйте, я Захар Юсупов!

Представляю свое решение к заданию ETL пайплайна .

# **Задание 1**








In [ ]:
# Импорт библиотек
import requests
import pandas as pd
import json
from datetime import datetime
import unittest

In [ ]:
# Функция для загрузки данных с API Open-Meteo
def extract_weather_data(url):
    """
    Загружает данные о погоде с API Open-Meteo.

    Args:
        url (str): URL API-запроса Open-Meteo.

    Returns:
        dict: JSON-данные о погоде или None в случае ошибки.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Ошибка при загрузке данных: {e}")
        return None

In [ ]:
# Функция для преобразования единиц измерения
def convert_units(df):
    """
    Преобразует единицы измерения в DataFrame:
    - Температура из °F в °C
    - Скорость ветра из узлов в м/с
    - Осадки из дюймов в мм
    - Видимость из футов в метры

    Args:
        df (pd.DataFrame): DataFrame с почасовыми данными.

    Returns:
        pd.DataFrame: Преобразованный DataFrame.
    """
    df = df.copy()
    # Температура: °F в °C: (°F - 32) * 5/9
    temp_columns = ['temperature_2m', 'dew_point_2m', 'apparent_temperature',
                    'temperature_80m', 'temperature_120m',
                    'soil_temperature_0cm', 'soil_temperature_6cm']
    for col in temp_columns:
        df[col] = (df[col] - 32) * 5/9

    # Скорость ветра: узлы в м/с (1 узел = 0.514444 м/с)
    df['wind_speed_10m'] *= 0.514444
    df['wind_speed_80m'] *= 0.514444

    # Осадки: дюймы в мм (1 дюйм = 25.4 мм)
    df['rain'] *= 25.4
    df['showers'] *= 25.4
    df['snowfall'] *= 25.4

    # Видимость: футы в метры (1 фут = 0.3048 метра)
    df['visibility'] *= 0.3048

    return df


In [ ]:
# Функция для преобразования Unix-времени в ISO 8601
def unix_to_iso(unix_time):
    """
    Преобразует Unix-время в формат ISO 8601.

    Args:
        unix_time (int): Unix-время в секундах.

    Returns:
        str: Время в формате ISO 8601.
    """
    return datetime.utcfromtimestamp(unix_time).isoformat() + 'Z'

# Основная функция трансформации данных
def transform_weather_data(data):
    """
    Трансформирует данные о погоде в требуемый формат.

    Args:
        data (dict): JSON-данные от Open-Meteo.

    Returns:
        pd.DataFrame: Трансформированный DataFrame в формате итоговой таблицы.
    """
    # Извлечение почасовых и ежедневных данных
    hourly_data = pd.DataFrame(data['hourly'])
    daily_data = pd.DataFrame(data['daily'])

    # Преобразование Unix-времени в ISO 8601
    hourly_data['time'] = pd.to_datetime(hourly_data['time'], unit='s', utc=True)
    daily_data['time'] = pd.to_datetime(daily_data['time'], unit='s', utc=True)
    daily_data['sunrise'] = daily_data['sunrise'].apply(unix_to_iso)
    daily_data['sunset'] = daily_data['sunset'].apply(unix_to_iso)

    # Преобразование единиц измерения
    hourly_data = convert_units(hourly_data)

    # Преобразование продолжительности светового дня в часы
    daily_data['daylight_hours'] = daily_data['daylight_duration'] / 3600

    # Создание итогового DataFrame
    result = []

    for date in daily_data['time']:
        # Выборка почасовых данных за день
        day_data = hourly_data[(hourly_data['time'].dt.date == date.date())]

        # Выборка данных за световой день
        sunrise = daily_data.loc[daily_data['time'] == date, 'sunrise'].iloc[0]
        sunset = daily_data.loc[daily_data['time'] == date, 'sunset'].iloc[0]
        sunrise_dt = pd.to_datetime(sunrise)
        sunset_dt = pd.to_datetime(sunset)
        daylight_data = day_data[(day_data['time'] >= sunrise_dt) &
                               (day_data['time'] <= sunset_dt)]

        # Вычисление средних значений за 24 часа
        avg_values_24h = day_data[[
            'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
            'apparent_temperature', 'temperature_80m', 'temperature_120m',
            'wind_speed_10m', 'wind_speed_80m', 'visibility'
        ]].mean().to_dict()

        # Вычисление средних значений за световой день
        avg_values_daylight = daylight_data[[
            'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
            'apparent_temperature', 'temperature_80m', 'temperature_120m',
            'wind_speed_10m', 'wind_speed_80m', 'visibility'
        ]].mean().to_dict()

        # Вычисление суммарных осадков
        total_precip_24h = day_data[['rain', 'showers', 'snowfall']].sum().to_dict()
        total_precip_daylight = daylight_data[['rain', 'showers', 'snowfall']].sum().to_dict()

        # Создание записи для текущего дня
        row = {
            'date': date.date(),
            'avg_temperature_2m_24h': avg_values_24h['temperature_2m'],
            'avg_relative_humidity_2m_24h': avg_values_24h['relative_humidity_2m'],
            'avg_dew_point_2m_24h': avg_values_24h['dew_point_2m'],
            'avg_apparent_temperature_24h': avg_values_24h['apparent_temperature'],
            'avg_temperature_80m_24h': avg_values_24h['temperature_80m'],
            'avg_temperature_120m_24h': avg_values_24h['temperature_120m'],
            'avg_wind_speed_10m_24h': avg_values_24h['wind_speed_10m'],
            'avg_wind_speed_80m_24h': avg_values_24h['wind_speed_80m'],
            'avg_visibility_24h': avg_values_24h['visibility'],
            'total_rain_24h': total_precip_24h['rain'],
            'total_showers_24h': total_precip_24h['showers'],
            'total_snowfall_24h': total_precip_24h['snowfall'],
            'avg_temperature_2m_daylight': avg_values_daylight.get('temperature_2m', None),
            'avg_relative_humidity_2m_daylight': avg_values_daylight.get('relative_humidity_2m', None),
            'avg_dew_point_2m_daylight': avg_values_daylight.get('dew_point_2m', None),
            'avg_apparent_temperature_daylight': avg_values_daylight.get('apparent_temperature', None),
            'avg_temperature_80m_daylight': avg_values_daylight.get('temperature_80m', None),
            'avg_temperature_120m_daylight': avg_values_daylight.get('temperature_120m', None),
            'avg_wind_speed_10m_daylight': avg_values_daylight.get('wind_speed_10m', None),
            'avg_wind_speed_80m_daylight': avg_values_daylight.get('wind_speed_80m', None),
            'avg_visibility_daylight': avg_values_daylight.get('visibility', None),
            'total_rain_daylight': total_precip_daylight.get('rain', None),
            'total_showers_daylight': total_precip_daylight.get('showers', None),
            'total_snowfall_daylight': total_precip_daylight.get('snowfall', None),
            'daylight_hours': daily_data.loc[daily_data['time'] == date, 'daylight_hours'].iloc[0],
            'sunrise_iso': daily_data.loc[daily_data['time'] == date, 'sunrise'].iloc[0],
            'sunset_iso': daily_data.loc[daily_data['time'] == date, 'sunset'].iloc[0]
        }
        result.append(row)

    return pd.DataFrame(result)



In [ ]:
# Функция для сохранения данных в CSV
def load_to_csv(df, filename='weather_forecast.csv'):
    """
    Сохраняет DataFrame в CSV-файл.

    Args:
        df (pd.DataFrame): DataFrame для сохранения.
        filename (str): Имя CSV-файла.
    """
    df.to_csv(filename, index=False)
    print(f"Данные сохранены в {filename}")

In [ ]:
# Основная функция ETL
def run_etl_pipeline(url):
    """
    Выполняет ETL-пайплайн: загрузка, трансформация и сохранение данных.

    Args:
        url (str): URL API-запроса Open-Meteo.
    """
    # Extract
    data = extract_weather_data(url)
    if data is None:
        return

    # Transform
    df = transform_weather_data(data)

    # Load
    load_to_csv(df)

In [ ]:
# Тесты
class TestETLPipeline(unittest.TestCase):
    def test_extract_weather_data(self):
        url = "https://api.open-meteo.com/v1/forecast?latitude=55.0344&longitude=82.9434&daily=sunrise,sunset,daylight_duration&hourly=temperature_2m&timezone=auto&start_date=2025-05-16&end_date=2025-05-16"
        data = extract_weather_data(url)
        self.assertIsNotNone(data)
        self.assertIn('hourly', data)
        self.assertIn('daily', data)

    def test_convert_units(self):
        df = pd.DataFrame({
            'temperature_2m': [32.0],
            'wind_speed_10m': [1.0],
            'visibility': [1000.0],
            'rain': [1.0]
        })
        converted = convert_units(df)
        self.assertAlmostEqual(converted['temperature_2m'][0], 0.0) # 32°F = 0°C
        self.assertAlmostEqual(converted['wind_speed_10m'][0], 0.514444)
        self.assertAlmostEqual(converted['visibility'][0], 304.8)
        self.assertAlmostEqual(converted['rain'][0], 25.4)

    def test_unix_to_iso(self):
        unix_time = 1747347493
        iso_time = unix_to_iso(unix_time)
        self.assertEqual(iso_time, '2025-05-16T04:38:13Z')

In [ ]:
# Запуск ETL-пайплайна
if __name__ == "__main__":
    url = ("https://api.open-meteo.com/v1/forecast?latitude=55.0344&longitude=82.9434"
           "&daily=sunrise,sunset,daylight_duration"
           "&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,"
           "apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,"
           "wind_speed_80m,wind_direction_10m,wind_direction_80m,visibility,"
           "evapotranspiration,weather_code,soil_temperature_0cm,soil_temperature_6cm,"
           "rain,showers,snowfall"
           "&timezone=auto&timeformat=unixtime&wind_speed_unit=kn"
           "&temperature_unit=fahrenheit&precipitation_unit=inch"
           "&start_date=2025-05-16&end_date=2025-05-30")

    run_etl_pipeline(url)

    # Запуск тестов
    unittest.main(argv=[''], exit=False)


 **Задание 2** Выберем PostgreSQL как базу данных, так как она широко используется, поддерживает сложные запросы и хорошо интегрируется с Python через библиотеку psycopg2




 **Задание 3** Выгрузка по запросуМодификация ETL для поддержки временного интервала

Функция run_etl_pipeline была изменена для принятия параметров start_date и end_date, которые задают временной интервал для запроса данных через API Open-Meteo. Параметры передаются в URL в формате YYYY-MM-DD. Также добавлены флаги save_to_csv и save_to_db для гибкого выбора способа выгрузки.



In [ ]:
#f"&start_date={start_date}&end_date={end_date}"

Задание 4  -не сделал пока что :(

Обновлённый код ETL с учетом SQL базы



In [ ]:
# ETL-пайплайн для обработки данных прогноза погоды с Open-Meteo
# Цель: Загрузить данные, трансформировать и сохранить в CSV и PostgreSQL
# Автор: [Ваше имя]
# Дата: 25.07.2025

# Импорт библиотек
import requests
import pandas as pd
import json
from datetime import datetime
import psycopg2
from psycopg2 import sql
import unittest

# Функция для загрузки данных с API Open-Meteo
def extract_weather_data(url):
    """
    Загружает данные о погоде с API Open-Meteo.

    Args:
        url (str): URL API-запроса Open-Meteo.

    Returns:
        dict: JSON-данные о погоде или None в случае ошибки.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Ошибка при загрузке данных: {e}")
        return None

# Функция для преобразования единиц измерения
def convert_units(df):
    """
    Преобразует единицы измерения в DataFrame:
    - Температура из °F в °C
    - Скорость ветра из узлов в м/с
    - Осадки из дюймов в мм
    - Видимость из футов в метры

    Args:
        df (pd.DataFrame): DataFrame с почасовыми данными.

    Returns:
        pd.DataFrame: Преобразованный DataFrame.
    """
    df = df.copy()
    temp_columns = ['temperature_2m', 'dew_point_2m', 'apparent_temperature',
                    'temperature_80m', 'temperature_120m',
                    'soil_temperature_0cm', 'soil_temperature_6cm']
    for col in temp_columns:
        df[col] = (df[col] - 32) * 5/9

    df['wind_speed_10m'] *= 0.514444
    df['wind_speed_80m'] *= 0.514444
    df['rain'] *= 25.4
    df['showers'] *= 25.4
    df['snowfall'] *= 25.4
    df['visibility'] *= 0.3048

    return df

# Функция для преобразования Unix-времени в ISO 8601
def unix_to_iso(unix_time):
    """
    Преобразует Unix-время в формат ISO 8601.

    Args:
        unix_time (int): Unix-время в секундах.

    Returns:
        str: Время в формате ISO 8601.
    """
    return datetime.utcfromtimestamp(unix_time).isoformat() + 'Z'

# Функция трансформации данных
def transform_weather_data(data):
    """
    Трансформирует данные о погоде в требуемый формат.

    Args:
        data (dict): JSON-данные от Open-Meteo.

    Returns:
        pd.DataFrame: Трансформированный DataFrame.
    """
    hourly_data = pd.DataFrame(data['hourly'])
    daily_data = pd.DataFrame(data['daily'])

    hourly_data['time'] = pd.to_datetime(hourly_data['time'], unit='s', utc=True)
    daily_data['time'] = pd.to_datetime(daily_data['time'], unit='s', utc=True)
    daily_data['sunrise'] = daily_data['sunrise'].apply(unix_to_iso)
    daily_data['sunset'] = daily_data['sunset'].apply(unix_to_iso)

    hourly_data = convert_units(hourly_data)
    daily_data['daylight_hours'] = daily_data['daylight_duration'] / 3600

    result = []
    for date in daily_data['time']:
        day_data = hourly_data[(hourly_data['time'].dt.date == date.date())]
        sunrise = daily_data.loc[daily_data['time'] == date, 'sunrise'].iloc[0]
        sunset = daily_data.loc[daily_data['time'] == date, 'sunset'].iloc[0]
        sunrise_dt = pd.to_datetime(sunrise)
        sunset_dt = pd.to_datetime(sunset)
        daylight_data = day_data[(day_data['time'] >= sunrise_dt) &
                               (day_data['time'] <= sunset_dt)]

        avg_values_24h = day_data[[
            'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
            'apparent_temperature', 'temperature_80m', 'temperature_120m',
            'wind_speed_10m', 'wind_speed_80m', 'visibility'
        ]].mean().to_dict()

        avg_values_daylight = daylight_data[[
            'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
            'apparent_temperature', 'temperature_80m', 'temperature_120m',
            'wind_speed_10m', 'wind_speed_80m', 'visibility'
        ]].mean().to_dict()

        total_precip_24h = day_data[['rain', 'showers', 'snowfall']].sum().to_dict()
        total_precip_daylight = daylight_data[['rain', 'showers', 'snowfall']].sum().to_dict()

        row = {
            'date': date.date(),
            'avg_temperature_2m_24h': avg_values_24h['temperature_2m'],
            'avg_relative_humidity_2m_24h': avg_values_24h['relative_humidity_2m'],
            'avg_dew_point_2m_24h': avg_values_24h['dew_point_2m'],
            'avg_apparent_temperature_24h': avg_values_24h['apparent_temperature'],
            'avg_temperature_80m_24h': avg_values_24h['temperature_80m'],
            'avg_temperature_120m_24h': avg_values_24h['temperature_120m'],
            'avg_wind_speed_10m_24h': avg_values_24h['wind_speed_10m'],
            'avg_wind_speed_80m_24h': avg_values_24h['wind_speed_80m'],
            'avg_visibility_24h': avg_values_24h['visibility'],
            'total_rain_24h': total_precip_24h['rain'],
            'total_showers_24h': total_precip_24h['showers'],
            'total_snowfall_24h': total_precip_24h['snowfall'],
            'avg_temperature_2m_daylight': avg_values_daylight.get('temperature_2m', None),
            'avg_relative_humidity_2m_daylight': avg_values_daylight.get('relative_humidity_2m', None),
            'avg_dew_point_2m_daylight': avg_values_daylight.get('dew_point_2m', None),
            'avg_apparent_temperature_daylight': avg_values_daylight.get('apparent_temperature', None),
            'avg_temperature_80m_daylight': avg_values_daylight.get('temperature_80m', None),
            'avg_temperature_120m_daylight': avg_values_daylight.get('temperature_120m', None),
            'avg_wind_speed_10m_daylight': avg_values_daylight.get('wind_speed_10m', None),
            'avg_wind_speed_80m_daylight': avg_values_daylight.get('wind_speed_80m', None),
            'avg_visibility_daylight': avg_values_daylight.get('visibility', None),
            'total_rain_daylight': total_precip_daylight.get('rain', None),
            'total_showers_daylight': total_precip_daylight.get('showers', None),
            'total_snowfall_daylight': total_precip_daylight.get('snowfall', None),
            'daylight_hours': daily_data.loc[daily_data['time'] == date, 'daylight_hours'].iloc[0],
            'sunrise_iso': daily_data.loc[daily_data['time'] == date, 'sunrise'].iloc[0],
            'sunset_iso': daily_data.loc[daily_data['time'] == date, 'sunset'].iloc[0]
        }
        result.append(row)

    return pd.DataFrame(result)

# Функция для сохранения в CSV
def load_to_csv(df, filename='weather_forecast.csv'):
    """
    Сохраняет DataFrame в CSV-файл.

    Args:
        df (pd.DataFrame): DataFrame для сохранения.
        filename (str): Имя CSV-файла.
    """
    df.to_csv(filename, index=False)
    print(f"Данные сохранены в {filename}")

# Функция для вставки в PostgreSQL
def load_to_postgres(df, db_params):
    """
    Вставляет данные в таблицу PostgreSQL, избегая дубликатов.

    Args:
        df (pd.DataFrame): DataFrame для вставки.
        db_params (dict): Параметры подключения к PostgreSQL.
    """
    try:
        conn = psycopg2.connect(**db_params)
        cursor = conn.cursor()

        insert_query = """
        INSERT INTO weather_data (
            date, avg_temperature_2m_24h, avg_relative_humidity_2m_24h,
            avg_dew_point_2m_24h, avg_apparent_temperature_24h,
            avg_temperature_80m_24h, avg_temperature_120m_24h,
            avg_wind_speed_10m_24h, avg_wind_speed_80m_24h, avg_visibility_24h,
            total_rain_24h, total_showers_24h, total_snowfall_24h,
            avg_temperature_2m_daylight, avg_relative_humidity_2m_daylight,
            avg_dew_point_2m_daylight, avg_apparent_temperature_daylight,
            avg_temperature_80m_daylight, avg_temperature_120m_daylight,
            avg_wind_speed_10m_daylight, avg_wind_speed_80m_daylight,
            avg_visibility_daylight, total_rain_daylight, total_showers_daylight,
            total_snowfall_daylight, daylight_hours, sunrise_iso, sunset_iso
        ) VALUES (
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
            %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
        ) ON CONFLICT (date) DO NOTHING
        """

        for _, row in df.iterrows():
            cursor.execute(insert_query, (
                row['date'], row['avg_temperature_2m_24h'],
                row['avg_relative_humidity_2m_24h'], row['avg_dew_point_2m_24h'],
                row['avg_apparent_temperature_24h'], row['avg_temperature_80m_24h'],
                row['avg_temperature_120m_24h'], row['avg_wind_speed_10m_24h'],
                row['avg_wind_speed_80m_24h'], row['avg_visibility_24h'],
                row['total_rain_24h'], row['total_showers_24h'], row['total_snowfall_24h'],
                row['avg_temperature_2m_daylight'], row['avg_relative_humidity_2m_daylight'],
                row['avg_dew_point_2m_daylight'], row['avg_apparent_temperature_daylight'],
                row['avg_temperature_80m_daylight'], row['avg_temperature_120m_daylight'],
                row['avg_wind_speed_10m_daylight'], row['avg_wind_speed_80m_daylight'],
                row['avg_visibility_daylight'], row['total_rain_daylight'],
                row['total_showers_daylight'], row['total_snowfall_daylight'],
                row['daylight_hours'], row['sunrise_iso'], row['sunset_iso']
            ))

        conn.commit()
        print("Данные успешно вставлены в PostgreSQL")

    except psycopg2.Error as e:
        print(f"Ошибка при вставке в PostgreSQL: {e}")
    finally:
        cursor.close()
        conn.close()

# Модифицированная функция ETL с поддержкой временного интервала
def run_etl_pipeline(start_date, end_date, latitude=55.0344, longitude=82.9434,
                     save_to_csv=True, save_to_db=False, db_params=None):
    """
    Выполняет ETL-пайплайн: загрузка, трансформация и сохранение данных.

    Args:
        start_date (str): Начальная дата в формате YYYY-MM-DD.
        end_date (str): Конечная дата в формате YYYY-MM-DD.
        latitude (float): Широта местоположения.
        longitude (float): Долгота местоположения.
        save_to_csv (bool): Сохранять в CSV или нет.
        save_to_db (bool): Сохранять в PostgreSQL или нет.
        db_params (dict): Параметры подключения к PostgreSQL.
    """
    # Формирование URL с заданным интервалом
    url = (
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}"
        "&daily=sunrise,sunset,daylight_duration"
        "&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,"
        "apparent_temperature,temperature_80m,temperature_120m,wind_speed_10m,"
        "wind_speed_80m,wind_direction_10m,wind_direction_80m,visibility,"
        "evapotranspiration,weather_code,soil_temperature_0cm,soil_temperature_6cm,"
        "rain,showers,snowfall"
        "&timezone=auto&timeformat=unixtime&wind_speed_unit=kn"
        "&temperature_unit=fahrenheit&precipitation_unit=inch"
        f"&start_date={start_date}&end_date={end_date}"
    )

    # Extract
    data = extract_weather_data(url)
    if data is None:
        return

    # Transform
    df = transform_weather_data(data)

    # Load
    if save_to_csv:
        load_to_csv(df, f'weather_forecast_{start_date}_to_{end_date}.csv')

    if save_to_db and db_params:
        load_to_postgres(df, db_params)

# Тесты
class TestETLPipeline(unittest.TestCase):
    def test_extract_weather_data(self):
        url = "https://api.open-meteo.com/v1/forecast?latitude=55.0344&longitude=82.9434&daily=sunrise,sunset,daylight_duration&hourly=temperature_2m&timezone=auto&start_date=2025-05-16&end_date=2025-05-16"
        data = extract_weather_data(url)
        self.assertIsNotNone(data)
        self.assertIn('hourly', data)
        self.assertIn('daily', data)

    def test_convert_units(self):
        df = pd.DataFrame({
            'temperature_2m': [32.0],
            'wind_speed_10m': [1.0],
            'visibility': [1000.0],
            'rain': [1.0]
        })
        converted = convert_units(df)
        self.assertAlmostEqual(converted['temperature_2m'][0], 0.0)
        self.assertAlmostEqual(converted['wind_speed_10m'][0], 0.514444)
        self.assertAlmostEqual(converted['visibility'][0], 304.8)
        self.assertAlmostEqual(converted['rain'][0], 25.4)

    def test_unix_to_iso(self):
        unix_time = 1747347493
        iso_time = unix_to_iso(unix_time)
        self.assertEqual(iso_time, '2025-05-16T04:38:13Z')

    def test_load_to_postgres(self):
        db_params = {
            'dbname': 'weather_forecast',
            'user': 'user',
            'password': 'password',
            'host': 'localhost',
            'port': '5432'
        }
        df = pd.DataFrame({
            'date': ['2025-05-16'],
            'avg_temperature_2m_24h': [12.3],
            'avg_relative_humidity_2m_24h': [45.2],
            'avg_dew_point_2m_24h': [5.0],
            'avg_apparent_temperature_24h': [10.0],
            'avg_temperature_80m_24h': [11.0],
            'avg_temperature_120m_24h': [10.5],
            'avg_wind_speed_10m_24h': [2.0],
            'avg_wind_speed_80m_24h': [3.0],
            'avg_visibility_24h': [10000.0],
            'total_rain_24h': [0.0],
            'total_showers_24h': [0.0],
            'total_snowfall_24h': [0.0],
            'avg_temperature_2m_daylight': [13.0],
            'avg_relative_humidity_2m_daylight': [40.0],
            'avg_dew_point_2m_daylight': [4.0],
            'avg_apparent_temperature_daylight': [11.0],
            'avg_temperature_80m_daylight': [12.0],
            'avg_temperature_120m_daylight': [11.5],
            'avg_wind_speed_10m_daylight': [2.5],
            'avg_wind_speed_80m_daylight': [3.5],
            'avg_visibility_daylight': [12000.0],
            'total_rain_daylight': [0.0],
            'total_showers_daylight': [0.0],
            'total_snowfall_daylight': [0.0],
            'daylight_hours': [16.5],
            'sunrise_iso': ['2025-05-16T04:38:13Z'],
            'sunset_iso': ['2025-05-16T21:06:29Z']
        })
        try:
            load_to_postgres(df, db_params)
        except psycopg2.Error:
            self.fail("Ошибка при вставке в PostgreSQL")

# Запуск ETL-пайплайна
if __name__ == "__main__":
    db_params = {
        'dbname': 'weather_forecast',
        'user': 'user',
        'password': 'password',
        'host': 'localhost',
        'port': '5432'
    }

    # Пример запуска с заданным интервалом
    run_etl_pipeline(
        start_date='2025-05-16',
        end_date='2025-05-30',
        save_to_csv=True,
        save_to_db=True,
        db_params=db_params
    )

    # Запуск тестов
    unittest.main(argv=[''], exit=False)